# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import edurel.utils.duckdb as ddbu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


In [ ]:
# con.close()

In [5]:
con = ddbu.con_file(f"{DB_DIR}/adw-oltp/adw-oltp.duckdb")

In [ ]:
con = ddbu.con_mem()
SQL_DIR = f"{DB_DIR}/company_en"
ddbu.exe_sql_file(con, f"{SQL_DIR}/schema.sql")
ddbu.exe_sql_file(con, f"{SQL_DIR}/data.sql")

In [ ]:
ddbu.tablenames(con)

['EmpProj', 'Employee', 'OrgUnit', 'Project']

In [ ]:
ddbu.columns(con, "OrgUnit")

[{'columnname': 'OUID', 'type': 'INTEGER', 'nullable': False},
 {'columnname': 'Head', 'type': 'INTEGER', 'nullable': True},
 {'columnname': 'SuperUnit', 'type': 'INTEGER', 'nullable': True},
 {'columnname': 'Name', 'type': 'VARCHAR', 'nullable': False}]

In [ ]:
ddbu.primary_keys(con, "OrgUnit")

['OUID']

In [ ]:
fks = ddbu.foreign_keys(con, "OrgUnit")
fks.append(ddbu.to_fk_dict('Employee|Head|EID'))
fks.append(ddbu.to_fk_dict('OrgUnit|SuperUnit|OUID'))
fks

[{'Employee': [['Head'], ['EID']]}, {'OrgUnit': [['SuperUnit'], ['OUID']]}]

In [7]:
additional_fks = {"OrgUnit": ["Employee|Head|EID", "OrgUnit|SuperUnit|OUID"]}
additional_fks = {}
ddbu.schema_yaml_print(con, additional_fks)

tables:
- tablename: Address
  columns:
  - columnname: AddressID
    type: INTEGER
    nullable: false
  - columnname: AddressLine1
    type: VARCHAR
    nullable: false
  - columnname: AddressLine2
    type: VARCHAR
    nullable: true
  - columnname: City
    type: VARCHAR
    nullable: false
  - columnname: StateProvinceID
    type: INTEGER
    nullable: false
  - columnname: PostalCode
    type: VARCHAR
    nullable: false
  - columnname: SpatialLocation
    type: VARCHAR
    nullable: true
  - columnname: rowguid
    type: VARCHAR
    nullable: true
  - columnname: ModifiedDate
    type: TIMESTAMP
    nullable: false
  primary_key:
  - AddressID
- tablename: AddressType
  columns:
  - columnname: AddressTypeID
    type: INTEGER
    nullable: false
  - columnname: Name
    type: VARCHAR
    nullable: false
  - columnname: rowguid
    type: VARCHAR
    nullable: true
  - columnname: ModifiedDate
    type: TIMESTAMP
    nullable: false
  primary_key:
  - AddressTypeID
- tablename: Bi